<a href="https://colab.research.google.com/github/david-c-brown/deep-chess/blob/main/deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import chess
import numpy as np
import itertools
import tensorflow as tf
import concurrent.futures
import time
import random
import math
from IPython.display import clear_output
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras import layers, mixed_precision
from tensorflow.keras.losses import Huber

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
     

In [4]:
# Board and Input Conversion Functions
def board_to_input(board, history=1):
    # Map piece characters to integers for encoding
    pieces = {
        "P": 0,
        "N": 1,
        "B": 2,
        "R": 3,
        "Q": 4,
        "K": 5,
        "p": 6,
        "n": 7,
        "b": 8,
        "r": 9,
        "q": 10,
        "k": 11,
    }
    # Initialize input tensor
    input_tensor = np.zeros((1, 8, 8, 12 * history), dtype=np.float32)

    # Create list of board states for input tensor
    board_states = [board]
    for _ in range(history - 1):
        if len(board.move_stack) > 0:
            move = board.pop()
            new_board = board.copy()
            board.push(move)
            board_states.insert(0, new_board)
        else:
            board_states.insert(0, None)

    # Encode pieces in input tensor
    for h, state in enumerate(board_states):
        if state is not None:
            for i in range(8):
                for j in range(8):
                    piece = state.piece_at(chess.square(i, j))
                    if piece:
                        input_tensor[0, i, j, pieces[str(piece)] + 12 * h] = 1

    return input_tensor

def moves_to_array(moves):
    # Create numpy array for legal moves
    move_array = np.zeros(4672)
    # encode moves in array
    for move in moves:
        move_idx = move.from_square * 73 + move.to_square
        move_array[move_idx] = 1
    return move_array

def array_to_move(board, move_array):
    # Get list of legal moves for board
    legal_moves = list(board.legal_moves)
    
    # Create numpy array for legal move probabilities
    legal_move_probs = np.zeros(len(legal_moves))

    # Map move array to legal move probabilities
    for i, move in enumerate(legal_moves):
        move_idx = move.from_square * 73 + move.to_square
        legal_move_probs[i] = move_array[move_idx]

    # Get legal move with highest probability
    best_move_idx = np.argmax(legal_move_probs)

    return legal_moves[best_move_idx] if legal_moves else None

# Evaluation and Backpropagation Functions
def backpropagate(node, value):
    # Update values of nodes in tree
    while node is not None:
        node.update(value)
        node = node.parent
        value = -value

# Memoize material balance function for performance
@lru_cache(maxsize=None)
def cached_evaluate_board(board_fen):
    # Create chess board from FEN string
    board = chess.Board(board_fen)
    return material_balance(board)

@tf.function(reduce_retracing=True)  
def model_predict(chess_model, board_input):
    # Convert input tensor to TensorFlow tensor
    board_input = tf.convert_to_tensor(board_input, dtype=tf.float32) 
    return chess_model(board_input) 

def evaluate(self, board):
    # Convert chess board to input tensor
    board_input = board_to_input(board).reshape(1, 8, 8, 12)
    move_probs, value_estimate = model_predict(self, board_input)
    return value_estimate.numpy().flatten()[0]

# Create chess board and get input tensor
board = chess.Board()
input_tensor = board_to_input(board)
print(input_tensor.shape)

(1, 8, 8, 12)


In [5]:
# Rewards
def material_balance(board):
    piece_values = {
        'P': 1,
        'N': 3,
        'B': 3,
        'R': 5,
        'Q': 9,
        'K': 0,
        'p': -1,
        'n': -3,
        'b': -3,
        'r': -5,
        'q': -9,
        'k': 0,
    }

    # Calculate material balance of board
    balance = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            balance += piece_values[str(piece)]

    return balance


def piece_mobility(board):
    mobility = 0
    for move in board.legal_moves:
        if board.turn:  
            mobility += 1
        else:
            mobility -= 1
    return mobility

def king_safety(board):
    safety = 0
    last_move = board.peek()  # Get the last move made

    if board.is_kingside_castling(last_move):
        safety += 15
    elif board.is_queenside_castling(last_move):
        safety += 10

    return safety

def center_control(board):
    center_squares = [chess.D4, chess.D5, chess.E4, chess.E5]
    control = 0

    for square in center_squares:
        attacker_color = board.turn
        defender_color = not board.turn
        if board.is_attacked_by(attacker_color, square):
            control += 1
        if board.is_attacked_by(defender_color, square):
            control -= 1
    return control



def is_double_pawn_push(board, move):
    piece = board.piece_at(move.from_square)
    if piece is None or piece.piece_type != chess.PAWN:
        return False
    return abs(move.from_square - move.to_square) == 16

def is_passed_pawn(board, move):
    piece = board.piece_at(move.from_square)
    if piece is None or piece.piece_type != chess.PAWN:
        return False

    # Check if there are opponent pawns in the adjacent files
    adjacent_files = [max(0, move.to_square % 8 - 1), min(7, move.to_square % 8 + 1)]
    direction = -8 if board.turn == chess.WHITE else 8

    for file in adjacent_files:
        for i in range(1, 6):
            adjacent_square = move.to_square + i * direction + file - move.to_square % 8
            if chess.square_is_valid(adjacent_square) and board.piece_at(adjacent_square) == chess.Piece(chess.PAWN, not board.turn):
                return False
    return True

def is_isolated_pawn(board, square, color):
    file = chess.square_file(square)
    rank = chess.square_rank(square)

    for r in range(8):
        if r == rank:
            continue
        if file > 0 and board.piece_at(chess.square(r, file - 1)) == chess.Piece(chess.PAWN, color):
            return False
        if file < 7 and board.piece_at(chess.square(r, file + 1)) == chess.Piece(chess.PAWN, color):
            return False
    return True

def pawn_structure(board):
    structure = 0
    isolated_pawns = 0
    last_move = board.peek()  # Get the last move made

    if is_double_pawn_push(board, last_move):
        structure -= 10
    elif is_passed_pawn(board, last_move):
        structure += 20
    isolated_pawns = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and piece.piece_type == chess.PAWN:
            if is_isolated_pawn(board, square, piece.color):
                isolated_pawns += 1 if piece.color == board.turn else -1
    structure -= 10 * isolated_pawns

    return structure




In [6]:
# MCTS Functions and Classes
class MCTSNode:
    # Initialize MCTSNode object
    def __init__(self, board, parent=None, move=None, prior=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.children = []
        self.visits = 0
        self.value = 0
        self.prior = prior
        self.additional_rewards = 0  
        self.eval_value = self.evaluate_board()

    def add_child(self, child_node):
        # Add child node to MCTSNode
        self.children.append(child_node)

    def update(self, value):
        self.visits += 1
        self.value += value

    def expand_children_parallel(self, chess_model, num_threads=4):
        # Expand children of MCTSNode in parallel using threads
        legal_moves = list(self.board.legal_moves)
        if len(legal_moves) == 0:
            return

        with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
            futures = [executor.submit(self._expand_child, move, chess_model) for move in legal_moves]

            for future in concurrent.futures.as_completed(futures):
                child = future.result()
                if child is not None:
                    self.children.append(child)

    def _expand_child(self, move, chess_model):
    # Create new child node with specified move and evaluate it
      child_board = self.board.copy()
      child_board.push(move)
      child_node = MCTSNode(child_board, self, move)
      child_node.evaluate(chess_model)
      rewards = 0
      rewards += material_balance(child_board)
      rewards += piece_mobility(child_board)
      rewards += king_safety(child_board)
      rewards += center_control(child_board)
      rewards += pawn_structure(child_board)

      child_node.additional_rewards = rewards
      
      return child_node


    def fully_expanded(self):
        # Check if MCTSNode is fully expanded
        return len(self.children) == len(list(self.board.legal_moves))

    def evaluate_board(self):
        return material_balance(self.board)

    def select_child(self, temperature):
        ucb_scores = [
            (child.value / (child.visits + 1e-10) +
             math.sqrt(2 * math.log(self.visits + 1e-10) / (child.visits + 1e-10)))
            for child in self.children
        ]

        # Calculate additional rewards for MCTSNode's child node
        for child in self.children:
            rewards = 0
            rewards += material_balance(child.board)
            rewards += piece_mobility(child.board)
            rewards += king_safety(child.board)
            rewards += center_control(child.board)
            rewards += pawn_structure(child.board)

            # Reward checkmate
            if child.board.is_checkmate():
                rewards += 100

            if child.board.is_insufficient_material() or child.board.can_claim_draw() or child.board.is_stalemate():
                rewards -= 50

            child.additional_rewards = rewards

        if temperature == 0:
            best_child_index = ucb_scores.index(max(ucb_scores))
        else:
            ucb_scores = [x**(1/temperature) for x in ucb_scores]
            total_score = sum(ucb_scores)
            probabilities = [x / total_score for x in ucb_scores]
            best_child_index = np.random.choice(len(self.children), p=probabilities)

        return self.children[best_child_index]

    def is_leaf(self):
        return len(self.children) == 0

    def evaluate(self, chess_model):
        # Evaluate the MCTSNode using the specified chess model
        self.visits += 1
        self.value = chess_model.evaluate(self.board)

    def best_child(self, c_param=1, eval_weight=0, temperature=1, noise=0):
        # Choose the best child of the MCTSNode
        if self.children == []:
            return None

        # Calculate the weights of all possible choices
        choices_weights = [
            ((c.value / (c.visits + 1e-8) + c_param * (c.prior + np.random.randn() * noise) * np.sqrt(self.visits) / (1 + c.visits) + eval_weight * c.eval_value) / temperature)
            for c in self.children
        ]

        return self.children[np.argmax(choices_weights)]

def expand(node, chess_model):
    # Expand the given node by creating child nodes for each legal move
    board_input = board_to_input(node.board).reshape(1, 8, 8, 12)
    move_probs, value_estimate = model_predict(chess_model, board_input)
    move_probs = move_probs.numpy().flatten()

    legal_moves = list(node.board.legal_moves)
    for move in legal_moves:
        # Create new child node for each legal move
        new_board = node.board.copy()
        new_board.push(move)
        move_idx = move.from_square * 73 + move.to_square
        prior = move_probs[move_idx]
        child_node = MCTSNode(new_board, parent=node, move=move, prior=prior)
        node.add_child(child_node)

    # Choose random child node to return
    return node.children[np.random.choice(len(node.children))]

def mcts(board, chess_model, num_simulations, temperature=1.0, noise=0.0, parallel_sims=True):
    # Run Monte Carlo Tree Search on the specified chess board and model
    root = MCTSNode(board, chess_model)

    # Run simulations sequentially or in parallel
    if not parallel_sims:
        for _ in range(num_simulations):
            current_board = board.copy()
            node = root
            while not node.is_leaf():
                node = node.select_child(temperature)
                current_board.push(node.move)
            if not current_board.is_game_over():
                node.expand_children(chess_model)
    else:
        def simulate(root):
            current_board = board.copy()
            node = root
            while not node.is_leaf():
                node = node.select_child(temperature)
                current_board.push(node.move)
            if not current_board.is_game_over():
                node.expand_children_parallel(chess_model)

        with ThreadPoolExecutor(max_workers=num_simulations) as executor:
            executor.map(simulate, [root]*num_simulations)
    
    best_child = root.best_child()
    # Return the move with the highest weight
    return best_child.move, best_child.additional_rewards
     

In [7]:
# Chess Model and Training Functions
def residual_block(inputs, num_filters, kernel_size=(3, 3)):
    # Defines a single residual block
    x = layers.Conv2D(num_filters, kernel_size, padding="same", activation=None)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(num_filters, kernel_size, padding="same", activation=None)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([inputs, x])
    x = layers.ReLU()(x)
    return x


def create_chess_model(num_res_blocks=19, num_filters=256):
    # Defines a deep residual network for playing chess
    input_shape = (8, 8, 12)

    # Input layer
    inputs = layers.Input(shape=input_shape, dtype=tf.float16)

    # Initial convolution layer
    x = layers.Conv2D(num_filters, kernel_size=(3, 3), padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)

    # Residual blocks
    for _ in range(num_res_blocks):
        x = residual_block(x, num_filters)

    # Policy head
    policy = layers.Conv2D(2, kernel_size=(1, 1), padding="same", activation="relu")(x)
    policy = layers.BatchNormalization()(policy)
    policy = layers.Flatten()(policy)
    policy = layers.Dense(4672, activation="softmax")(policy)

    # Value head
    value = layers.Conv2D(1, kernel_size=(1, 1), padding="same", activation="relu")(x)
    value = layers.BatchNormalization()(value)
    value = layers.Flatten()(value)
    value = layers.Dense(1, activation="tanh")(value)

    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=[policy, value])

    def evaluate(self, board):
        # Evaluates a board state using the model
        board_input = board_to_input(board).reshape(1, 8, 8, 12)
        move_probs, value_estimate = model_predict(self, board_input)
        return value_estimate.numpy().flatten()[0]

    # Add the evaluate method to the model object
    model.evaluate = evaluate.__get__(model)
    return model

def compute_advantages(rewards, values, gamma=0.99, lambda_=0.95):
    # Compute the generalized advantage estimate (GAE) from the rewards and values
    advantages = np.zeros_like(rewards)
    gae = 0
    for t in reversed(range(len(rewards))):
        delta = rewards[t] + gamma * values[t + 1] - values[t]
        gae = delta + gamma * lambda_ * gae
        advantages[t] = gae
    return advantages

def ppo_loss_fn(advantages, old_probs, actions, logits, values, clip_epsilon=0.2, value_loss_coeff=0.5, entropy_coeff=0.01):
    # Calculate the PPO loss function
    prob_ratio = tf.exp(tf.nn.log_softmax(logits) - tf.stop_gradient(tf.nn.log_softmax(old_probs)))
    prob_ratio = tf.reduce_sum(prob_ratio * actions, axis=-1)
    clipped_prob_ratio = tf.clip_by_value(prob_ratio, 1 - clip_epsilon, 1 + clip_epsilon)
    surrogate_loss = -tf.reduce_mean(tf.minimum(prob_ratio * advantages, clipped_prob_ratio * advantages))

    value_loss = Huber()(values, old_values)

    entropy_loss = -tf.reduce_mean(tf.reduce_sum(tf.nn.softmax(logits) * tf.nn.log_softmax(logits), axis=-1))

    total_loss = surrogate_loss + value_loss_coeff * value_loss - entropy_coeff * entropy_loss
    return total_loss

chess_model = create_chess_model()
chess_model.summary()
chess_model.evaluate = evaluate.__get__(chess_model)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8, 8, 12)]   0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 8, 8, 256)    27904       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 8, 8, 256)   1024        ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 8, 8, 256)    590080      ['batch_normalization[0][0]']

In [ ]:
# Create separate models for black and white players
white_chess_model = create_chess_model()
black_chess_model = create_chess_model()

white_weights_file = "/content/drive/MyDrive/deep_chess/white_chess_model_weights.h5"
black_weights_file = "/content/drive/MyDrive/deep_chess/black_chess_model_weights.h5"

try:
    # Try loading the saved weights for the white and black models
    white_chess_model.load_weights(white_weights_file)
    black_chess_model.load_weights(black_weights_file)
except:
    # If the weights file does not exist, do nothing
    pass

# Define the optimizer and loss functions for the chess models
optimizer = 'adam'
loss = 'categorical_crossentropy'

# Compile the models with the defined optimizer and loss
white_chess_model.compile(optimizer=optimizer, loss=loss)
black_chess_model.compile(optimizer=optimizer, loss=loss)

start_time = time.time()

def play_game(game_number, white_model, black_model, num_games, white_score, black_score,draws,num_simulations=100, verbose=False, time_per_move=2):
    board = chess.Board()
    running_reward_sum = 0

    white_rewards = 0
    black_rewards = 0

    while not board.is_game_over():
        if verbose:
            clear_output(wait=True)
            print(f"Game {game_number + 1} out of {num_games}. Reward total: {running_reward_sum}")
            print(f"White wins: {white_score[0]}, {white_rewards} | Black wins: {black_score[0]}, {black_rewards} | Draws: {draws[0]}")
            elapsed_time = (time.time() - start_time)/60
            print(f"Elapsed time: {elapsed_time:.2f}m")
            print(board)

        if board.turn == chess.WHITE:
            move, rewards = mcts(board, white_model, num_simulations=num_simulations)
            white_rewards += rewards
        else:
            move, rewards = mcts(board, black_model, num_simulations=num_simulations)
            black_rewards += rewards
        
        board.push(move)
        running_reward_sum += rewards

    result = board.result()

    return result, white_rewards, black_rewards


def run_multiple_games(white_chess_model, black_chess_model, num_games, num_simulations=100, num_threads=4, time_per_move=2):
    game_results = []
    white_score = [0] 
    black_score = [0]
    draws = [0] 

    white_rewards_list = []
    black_rewards_list = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(play_game, i, white_chess_model, black_chess_model, num_games, white_score, black_score, draws,num_simulations, True, time_per_move) for i in range(num_games)]

        for future in concurrent.futures.as_completed(futures):
            result, white_rewards, black_rewards = future.result()            
            game_results.append(result)
            white_rewards_list.append(white_rewards)
            black_rewards_list.append(black_rewards)

            if result == "1-0":
                white_score[0] += 1  
            elif result == "0-1":
                black_score[0] += 1 
            elif result == "1/2-1/2":
                draws[0] += 1
    
    white_rewards_array = np.array(white_rewards_list)
    black_rewards_array = np.array(black_rewards_list)
    results_array = np.array([1 if r == "1-0" else -1 if r == "0-1" else 0 for r in game_results])

    white_rewards_correlation = np.corrcoef(white_rewards_array, results_array)[0, 1]
    black_rewards_correlation = np.corrcoef(black_rewards_array, results_array)[0, 1]

    print(f"Correlation between white rewards and game outcomes: {white_rewards_correlation}")
    print(f"Correlation between black rewards and game outcomes: {black_rewards_correlation}")

    return game_results, white_score[0], black_score[0], draws[0], white_rewards_correlation, black_rewards_correlation

num_games = 10
game_results = run_multiple_games(white_chess_model, black_chess_model, num_games, num_simulations=50, num_threads=8, time_per_move=0.5)

# Save the weights of the trained models
white_chess_model.save_weights("/content/drive/MyDrive/deep_chess/white_chess_model_weights.h5")
black_chess_model.save_weights("/content/drive/MyDrive/deep_chess/black_chess_model_weights.h5")

Game 7 out of 10. Reward total: 844
White wins: 3, -262 | Black wins: 0, 1106 | Draws: 4
Elapsed time: 40.70m
. . . . . . . .
. . . . . R . K
. . . . . . . .
. . . . . . . .
P . . k . . . .
. . . . . . . .
. . . b . . b .
. . . . . . . .
